<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/neural_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Language Modeling

In [1]:
import os
import sys
sys.path.append('utils/')

In [17]:
import utils.ngram_utils as ngram_utils
from utils.amazon_dataset import AmazonDatase
from torch.utils.data import DataLoader
from utils.neural_lm import BagOfNGrams, DecoderMLP, seq2seq
import utils.global_variables as gl
import torch
import utils.amazon_dataset as ds

ImportError: cannot import name 'pad' from 'utils.amazon_dataset' (/home/roberta/ammi-2019-nlp/01-day-LM/utils/amazon_dataset.py)

In [3]:
torch.manual_seed(1)


In [4]:
# Read data from .txt files and create lists of reviews

train_data = []
# create a list of all the reviews 
with open('../data/amazon_reviews_clothing_train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../data/amazon_reviews_clothing_valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [5]:
len(train_data), len(valid_data)

(222919, 27869)

In [6]:
train_data = train_data[:100]
valid_data = valid_data[:10]

In [7]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)


In [8]:
gl.SOS_TOKEN
train_padded_uni = ngram_utils.pad_sentences(train_data_tokenized, 2)
train_padded_uni[0]

['<sos>',
 'this',
 'is',
 'a',
 'great',
 'tutu',
 'and',
 'at',
 'a',
 'really',
 'great',
 'price',
 '.',
 'it',
 'doesn',
 "'",
 't',
 'look',
 'cheap',
 'at',
 'all',
 '.',
 'i',
 "'",
 'm',
 'so',
 'glad',
 'i',
 'looked',
 'on',
 'amazon',
 'and',
 'found',
 'such',
 'an',
 'affordable',
 'tutu',
 'that',
 'isn',
 "'",
 't',
 'made',
 'poorly',
 '.',
 'a',
 '+',
 '+',
 '<eos>']

In [9]:
# n = 1
train_padded_uni = ngram_utils.pad_sentences(train_data_tokenized, 1)
train_unigram = ngram_utils.find_ngrams(train_padded_uni, 1)
train_vocab_unigram, train_count_unigram = ngram_utils.ngram_counts(train_unigram)
train_id2token_unigram, train_token2id_unigram = ngram_utils.ngram_dict(train_vocab_unigram)

valid_padded_uni = ngram_utils.pad_sentences(valid_data_tokenized, 1)
valid_unigram = ngram_utils.find_ngrams(valid_padded_uni, 1)
valid_vocab_unigram, count_unigram = ngram_utils.ngram_counts(valid_unigram)
valid_id2token_unigram, valid_token2id_unigram = ngram_utils.ngram_dict(valid_vocab_unigram)

In [10]:
train_id2token_unigram[:10]

['<pad>',
 '<unk>',
 '<sos>',
 '<eos>',
 ('.',),
 (',',),
 ('the',),
 ('i',),
 ('to',),
 ('and',)]

In [13]:
N = 10
train_data_id = ngram_utils.create_data_id(train_unigram, train_token2id_unigram)
train_data_id_merged = ngram_utils.create_data_id_merged(train_data_id, train_token2id_unigram, N)

valid_data_id = ngram_utils.create_data_id(valid_unigram, valid_token2id_unigram)
valid_data_id_merged = ngram_utils.create_data_id_merged(valid_data_id, valid_token2id_unigram, N)

In [14]:
train_data_id_merged[0]

([16, 14, 10, 65, 197, 9, 49, 10, 67, 65], 72)

In [15]:
train_dataset = AmazonDataset(train_data_id_merged, max_inp_length=None, use_cuda=True)
train_loader = DataLoader(train_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

valid_dataset = AmazonDataset(valid_data_id_merged, max_inp_length=None, use_cuda=True)
valid_loader = DataLoader(valid_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

NameError: name 'batchify' is not defined

In [ ]:
train_dataset, train_loader

In [ ]:
train_data_id_merged

In [ ]:
train_dataset[0], train_dataset[0][0].shape, \
valid_dataset[0], valid_dataset[0][0].shape

In [ ]:
num_epochs = 500
encoder = BagOfNGrams(len(train_id2token_unigram), emb_dim=300, hidden_size=256, out_size=128, activation='Tanh', nlayers=1, reduce='mean', dropout=0.0, batch_norm=False)
decoder = DecoderMLP(input_size=128, output_size=len(train_id2token_unigram), hidden_size=256)
model = seq2seq(encoder, decoder, use_cuda=False, lr=1e-1)

for epoch in range(num_epochs):
    # Train
    train_loss_epoch = 0
    for i, (data, labels) in _tqdm(enumerate(train_loader)):
        prediction, loss = model.train_step(data, labels)
        train_loss_epoch += loss
    if epoch % 10 == 0:
        print("Epoch {}: Loss {}".format(epoch, train_loss_epoch))
#         print("prediction ", prediction)

        generated = model.eval_step()
        generated_str = ' '.join([g[0] for g in generated])
        print("Generated Sentence: ", generated_str)
        
